In [ ]:
!pip install gurobipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import os
import sys
# sys.path.append('../')
sys.path.append('/content/gdrive/MyDrive/SVR_Codes/SVR_CG')
from Codes.model import *
from Codes.utils import *

plt.style.use('default')

from tqdm import tqdm
import pickle
sys.path.append('/content/gdrive/MyDrive/SVR_Codes/ExptsEval')
RESULT_DIR = "/content/gdrive/MyDrive/SVR_Codes/ExptsEval"

from timeit import default_timer
from sklearn.datasets import make_blobs




In [ ]:
def Blobs(N,D,K,box,seed,ratio_std_x = 0.01, ratio_std_y=0.0):


    generator = np.random.RandomState(seed)
    regParam = np.round(generator.uniform(-3,3, size=(K, D+1)),2)
    # print(regParam)
    # box = 1
    generator = np.random.RandomState(seed)
    centers = generator.uniform(0, box, size=(K, D))
    # print(centers)
    # ratio_std_x = 0.01
    # ratio_std_y = 0.0

    X, y = make_blobs(n_samples=N, centers=centers, n_features=D, cluster_std = ratio_std_x,
                        random_state=seed)
    # print(centers)
    cols = [ 'X' + str(i+1) for i in range(D) ]
    df_data = pd.DataFrame(X, columns = cols)

    for k in range(K):
        generator = np.random.RandomState(seed)
        df_data.loc[y==k,'Y'] = X[y == k] @ regParam[k,:-1] + regParam[k,-1] + generator.normal(0,ratio_std_y,X[y==k].shape[0])

    # df_data.plot('X1', 'Y', kind = 'scatter', s = 20)
    # plt.show()

    return df_data, X,y


## MILP with constraint generation 

In [ ]:
N_list = [100 , 315, 1000, 3150, 10000]#, 315000, 1000000]

K = 2
D_list = [1,2,5] 

box = 100
seed = 124

tol_list = [0.001,0.002,0.005]
initCons_list = [4,5,7]
cnt = 0
ratio_std_x=0.5
ratio_std_y=0.001

for i, D in enumerate(tqdm(D_list)):

    optgap_list=[]

    time_list = []
    metrics_list = []

    # obj_list = []
    constrsCnt_list = []

    optVal_list = []

    weights_list = []
    bias_list = []
    
    for N in tqdm(N_list):
        print('\n\n\nStarting N = %i K = %i and D = %i' %(N,K,D))

        # df_data, X,y = Blobs(N,D,K,box, seed)
        df_data, X,y = Blobs(N,D,K,box,seed,ratio_std_x, ratio_std_y)


        clus = CLR(K =K , f = D, max_iter= 20, compute=True, random_state=234)
        clus.set_model(Cl_SVR_Cg(outputFlag=False, initConstrCnt=initCons_list[i], optimalGap= 0.05, tol = tol_list[i], time = 10, initConstrIntRatio= 10, step_plots=False, outliersCnt = 0))


        clus.fit(df_data )

        metrics_list.append(clus.evaluate(y))
        # print(metrics_list[-1])
        time_list.append(clus.run_time)

        constrsCnt_list.append(len(clus.constr_gen_pts))
        optgap_list.append(clus.model_optGap)

        optVal_list.append(clus.model_optVal)

        weights_list.append(clus.weights)
        bias_list.append(clus.bias)


        # obj_list.append(clus)

        cnt+=1

        print('Finished running N = %i K = %i and D = %i' %(N,K,D))
        print('*'*100)




    with open(os.path.join(RESULT_DIR, 'SynCompare_'+str(K)+str(D)+'_clsvr.pkl'), 'wb') as outp:

        pickle.dump(metrics_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(time_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(constrsCnt_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(optgap_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(optVal_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(weights_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(bias_list, outp, pickle.HIGHEST_PROTOCOL)





# FULL MILP

In [ ]:
N_list = [100 , 315, 1000, 3150, 10000]#, 315000, 1000000]

K = 2
D_list = [1,2,5] 

box = 100
seed = 124


cnt = 0
ratio_std_x=0.5
ratio_std_y=0.001

for i, D in enumerate(tqdm(D_list)):

    optgap_list=[]

    time_list = []
    metrics_list = []

    # obj_list = []
    constrsCnt_list = []

    optVal_list = []

    weights_list = []
    bias_list = []
    
    for N in tqdm(N_list):
        print('\n\n\nStarting N = %i K = %i and D = %i' %(N,K,D))

        # df_data, X,y = Blobs(N,D,K,box, seed)
        df_data, X,y = Blobs(N,D,K,box,seed,ratio_std_x, ratio_std_y)


        clus = CLR(K =K , f = D, compute = True, random_state=243)
        clus.set_model(Cl_SVR_milp(outputFlag=False, WarmStart=True, optimalGap= 0.05, time = 120))

        clus.fit(df_data )

        metrics_list.append(clus.evaluate(y))
        # print(metrics_list[-1])
        time_list.append(clus.run_time)

        constrsCnt_list.append(clus.constr_gen_pts)
        optgap_list.append(clus.model_optGap)

        optVal_list.append(clus.model_optVal)

        weights_list.append(clus.weights)
        bias_list.append(clus.bias)


        # obj_list.append(clus)

        cnt+=1

        print('Finished running N = %i K = %i and D = %i' %(N,K,D))
        print('*'*100)




    with open(os.path.join(RESULT_DIR, 'SynCompare_'+str(K)+str(D)+'_fullmilp.pkl'), 'wb') as outp:

        pickle.dump(metrics_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(time_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(constrsCnt_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(optgap_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(optVal_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(weights_list, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(bias_list, outp, pickle.HIGHEST_PROTOCOL)



